# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [5]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards: 100%|██████████| 7/7 [00:04<00:00,  1.44it/s]


In [6]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [8]:
input_text = "解释下乾卦是什么？"

In [9]:
response, history = base_model.chat(tokenizer, query=input_text)

In [10]:
print(response)

乾卦是八卦之一，也是八宫图之一，它代表了天、干燥、刚强、积极、行动等含义。乾卦是由两个阴爻夹一个阳爻构成，象征着天宇的广阔和阳刚之气的存在。

在八宫图中，乾卦位于北方，与事业、努力、积极、刚强、独立等有关。它代表了一个积极向上的态度，提示着你应该勇往直前，充满自信地迎接挑战。同时，乾卦也提醒要维护自己的尊严和自尊，保持刚强和坚定的态度。

在易经中，乾卦的卦辞是“元、亨、利、贞”，这意味着它是一种正义、积极、正确的态度和行为。乾卦还与“八卦”中的“乾”有关，乾卦所代表的能量和态度是八卦中最为积极和阳性的。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [11]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它代表了诉讼、争端、矛盾、困扰等含义。讼卦是由两个阳爻夹一个阴爻构成，象征着阳刚之气的存在，但也暗示着矛盾和争端的存在。

在八宫图中，讼卦位于西南方，与法律、法规、公正、争端、矛盾等有关。它代表了一种公正、公平的态度，提示着你应该以法律为依据，坚持公正和正义，解决矛盾和争端。

在易经中，讼卦的卦辞是“元、亨、利、贞”，这意味着它是一种正义、正确、有益的态度和行为。讼卦也提醒要通过法律和公正来解决矛盾和争端，以维护社会秩序和公正。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [12]:
from peft import PeftModel, PeftConfig

epochs = 5
# timestamp = "20240118_164514"
timestamp = "20250814_163205"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch={epochs}, automade-dataset(fixed))-{timestamp}"

In [13]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [14]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [15]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [16]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦是一个卜卦，在周易卦象中，它由下卦乾和上卦坎组成。坎象征着云，乾象征着天，云聚于天，形成了云雾之间的景象。在这个卦象中，云雾即将消散，君子应当等待时机，观望天象，以行天下。', 'content': '\n师卦的解释预示着云雾即将消散，天象将明，君子观此卦象，等待时机，行天下。在事业方面，预示着 initial success，但必须警惕风险，及时止损，或 else将失去投资。对于经商，初期会有利，但务必要注意风险，不可轻率行事。\n\n师卦的核心哲学是：行事必须等待时机，不能急功近利，应观天象行事，随机应变。在事业和经商中，必须注意风险和机会，及时止损，等待时机，不能急功近利。'}


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（手动数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=50, handmade-dataset)

In [17]:
from peft import PeftModel, PeftConfig

handmade_epochs = 50
handmade_timestamp = "20250817_201445"
handmade_peft_model_path = f"models/{model_name_or_path}-epoch{handmade_epochs}-{handmade_timestamp}"

config = PeftConfig.from_pretrained(handmade_peft_model_path)
handmade_qlora_model_e3 = PeftModel.from_pretrained(base_model, handmade_peft_model_path)
handmade_training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, handmade_qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，代表一种强调积极、力量、刚健的哲学观念。乾卦由三个阳爻夹一个阴爻构成，象征天、强调象征力量和权威，代表宇宙中充满活力与前进的意象。在五行中，乾卦代表木，与象征归纳、组织、规划有关，强调经营与领导能力。


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 解释下乾卦是什么？ 乾卦象征天，象征象征，代表强大，启示启示，启示启示，启示启示，象征象征，象征象征象征象征象征，象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象

In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, handmade_qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦 象征地牢与水泽，象征坚毅与智慧，启示地水师卦象征坚毅与智慧，启示面对困境需坚定信念，依赖自己的智慧与力量。


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 地水师卦是什么？ 地水师卦象征地象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象征象

In [21]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, handmade_qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
周易中的讼卦（象征诉讼）是由乾卦和震卦组合而成，象征诉讼争议，强调双方争论不休，需要通过公正的审判来解决。这个卦提醒人们在处理争议时要有公正态度，遵循法律法规，避免冲突升级。


微调后（ChatGLM3-6B(Epoch=5, automade-dataset(fixed))-20250814_163205）：
[gMASK]sop 周易中的讼卦是什么？ 周易中的讼卦象征争议与矛盾，强调解决争端需要双方共同协商，强调争议的解决需要公正公平。
